In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import os
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from PIL import Image, ImageFile
import shutil

In [18]:
CARPETA_DATOS = "/content/drive/MyDrive/dataset"

setattr(tfds.image_classification.cats_vs_dogs, '_URL',
        "https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_5340.zip")

ds, info = tfds.load('cats_vs_dogs', split='train', as_supervised=True, with_info=True)
ds = ds.take(200)

os.makedirs(CARPETA_DATOS + "/Arnold", exist_ok=True)
os.makedirs(CARPETA_DATOS + "/Desconocido", exist_ok=True)


In [19]:
i = 0
for img, label in ds:
    img = Image.fromarray(img.numpy())
    img.save(f"{CARPETA_DATOS}/Desconocido/img_{i}.jpg")
    i += 1

ImageFile.LOAD_TRUNCATED_IMAGES = True

datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)

In [ ]:

train_generator = datagen.flow_from_directory(
    CARPETA_DATOS,
    target_size=(100, 100),
    batch_size=32,
    class_mode='binary',
    subset='training',
    classes=['Desconocido', 'Arnold']
)

validation_generator = datagen.flow_from_directory(
    CARPETA_DATOS,
    target_size=(100, 100),
    batch_size=32,
    class_mode='binary',
    subset='validation',
    classes=['Desconocido', 'Arnold']
)

In [21]:
modeloCNN = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(100, 100, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(100, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

modeloCNN.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

historial = modeloCNN.fit(
    train_generator,
    epochs=20,
    validation_data=validation_generator
)

modeloCNN.save('modelo_arnold.h5')

if not os.path.exists('carpeta_salida'):
    os.makedirs('carpeta_salida')

Epoch 1/20
9/9 ━━━━━━━━━━━━━━━━━━━━ 7s 637ms/step - accuracy: 0.4979 - loss: 0.7680 - val_accuracy: 0.6061 - val_loss: 0.6262
Epoch 2/20
9/9 ━━━━━━━━━━━━━━━━━━━━ 6s 673ms/step - accuracy: 0.6224 - loss: 0.6221 - val_accuracy: 0.9091 - val_loss: 0.4243
Epoch 3/20
9/9 ━━━━━━━━━━━━━━━━━━━━ 5s 562ms/step - accuracy: 0.8678 - loss: 0.3951 - val_accuracy: 0.8485 - val_loss: 0.2845
Epoch 4/20
9/9 ━━━━━━━━━━━━━━━━━━━━ 6s 666ms/step - accuracy: 0.9363 - loss: 0.1798 - val_accuracy: 0.9091 - val_loss: 0.1800
Epoch 5/20
9/9 ━━━━━━━━━━━━━━━━━━━━ 5s 580ms/step - accuracy: 0.9664 - loss: 0.1122 - val_accuracy: 0.9697 - val_loss: 0.1259
Epoch 6/20
9/9 ━━━━━━━━━━━━━━━━━━━━ 6s 622ms/step - accuracy: 0.9668 - loss: 0.0784 - val_accuracy: 0.9545 - val_loss: 0.1008
Epoch 7/20
9/9 ━━━━━━━━━━━━━━━━━━━━ 5s 634ms/step - accuracy: 0.9565 - loss: 0.0909 - val_accuracy: 0.9545 - val_loss: 0.2502
Epoch 8/20
9/9 ━━━━━━━━━━━━━━━━━━━━ 5s 565ms/step - accuracy: 0.9597 - loss: 0.0951 - val_accuracy: 0.9242 - val_loss:

In [22]:
!pip install tensorflowjs
!tensorflowjs_converter --input_format keras modelo_arnold.h5 carpeta_salida

2025-12-03 22:02:08.554038: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1764799328.567861   15885 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1764799328.571875   15885 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1764799328.582449   15885 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1764799328.582479   15885 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1764799328.582483   15885 computation_placer.cc:177] computation placer alr

In [25]:
from google.colab import files
files.download('carpeta_salida/model.json')
files.download('carpeta_salida/group1-shard1of2.bin')
files.download('carpeta_salida/group1-shard2of2.bin')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>